In [ ]:
! pip install transformers datasets
! pip install evaluate
! pip install sentence-transformers
! pip install tabulate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (1

In [ ]:
! pip install accelerate -U

  Using cached accelerate-0.30.1-py3-none-any.whl (302 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.21.0
    Uninstalling accelerate-0.21.0:
      Successfully uninstalled accelerate-0.21.0


### Προσοχή

Μη διαγράψετε τα # insert your code here σχόλια, καθώς βοηθούν στη διόρθωση. Συμπληρώστε τον κώδικά σας μετά από τα σχόλια αυτά.

# Imports

In [ ]:
from datasets import load_dataset, Dataset
import torch
import numpy as np
import pandas as pd
from tabulate import tabulate
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForMultipleChoice,
    AutoModelForSequenceClassification,
)
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

from warnings import simplefilter

simplefilter(action="ignore", category=FutureWarning)

# Μέρος Α: Fine-tune a pretrained model

Τα γλωσσικά μοντέλα αποτελούνται από δύο στάδια εκπαίδευσης:
1. Pre-training σε μεγάλα unlabelled datasets. Το pre-training είναι υπολογιστικά πολύ ακριβό και γι αυτό στην πράξη δε το χρησιμοποιούμε όταν θέλουμε να τρέξουμε ένα μοντέλο σε ένα καινούργιο dataset. Μπορούμε να σκεφτούμε το pre-training ως τη διαδικασία εκμάθησης γλωσσικών κανόνων κι εννοιών, οι οποίες στη συνέχεια μπορούν να χρησιμοποιηθούν για διάφορους σκοπούς.

2. Fine-tuning σε μικρότερα labelled datasets. Το fine-tuning πρακτικά εκμεταλλεύεται τις ιδιότητες του transfer learning προκειμένου να μεταφέρουμε τη 'γνώση' που έχει αποθηκευθεί στο γλωσσικό μοντέλο κατά τη διάρκεια του pre-training σε συγκεκριμένα task. Κάθε task εξυπηρετείται μέσω στοχευμένων datasets. Για παράδειγμα, κάποια datasets αναφέρονται στην ταξινόμηση κειμένων σε κατηγιορίες (text classification), άλλα datasets περιέχουν ερωτήσεις οι οποίες πρέπει να απαντηθούν (question answering) κι άλλα πολλά.

Κάποια κλασικά tasks της επεξεργασίας φυσικής γλώσσας είναι τα ακόλουθα:
- Text classification
- Question answering
- Natural language inference
- Fill mask
- Semantic similarity

Περισσότερες πληροφορίες μπορείτε να βρείτε στον ακόλουθο σύνδεσμο στο domain Natural Language Processing: https://huggingface.co/models

Στο πρώτο κομμάτι της παρούσας εργαστηριακής άσκησης, θα χρησιμοποιήσουμε το pre-training fine-tuning σενάριο για να ταξινομήσουμε reviews.

## Pipelines

Με τη χρήση του text-classification pipeline μπορούμε να τρέξουμε γλωσσικά μοντέλα που αφορούν tasks ταξινόμησης. Το natural language inference (NLI) task αποτελεί ένα task ταξινόμησης, αφού το σχετικό μοντέλο (εν προκειμένω το roberta-large-mnli) καλείται να ταξινομήσει ένα κείμενο σε μία από τις 3 κατηγορίες [entailment/neutral/contradiction].

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "roberta-large-mnli")
classifier("A soccer game with multiple males playing. Some men are playing a sport.")
## [{'label': 'ENTAILMENT', 'score': 0.98}]
```

Ένα άλλο task ταξινόμησης αφορά την αξιολόγηση του κατά πόσο ένα κείμενο είναι γραμματικά ορθό (acceptable) ή όχι (unacceptable):

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "textattack/distilbert-base-uncased-CoLA")
classifier("I will walk to home when I went through the bus.")
##  [{'label': 'unacceptable', 'score': 0.95}]
```

## Σύνολο δεδομένων Yelp polarity

Κατεβάζουμε το [Yelp Polarity](https://huggingface.co/datasets/yelp_polarity) dataset το οποίο περιέχει reviews που εκφράζουν συναισθήματα πελατών για εστιατόρια. Τα reviews αυτά χωρίζονται σε κατηγορίες, και ο σκοπός μας είναι να κατηγοριοποιήσουμε νέα reviews στις σωστές κατηγορίες.

In [ ]:
# insert your code here
dataset = load_dataset("fancyzhx/yelp_polarity")

Επειδή το σύνολο δεδομένων του Yelp Polarity περιέχει πολλά δείγματα, προκειμένου να επιταχύνουμε τη διαδικασία του fine-tuning συστήνουμε να διατηρήσετε 300 δείγματα από το train set και 300 δείγματα από το test set.

Ελέγξτε τον αριθμό κατηγοριών που υπάρχουν συνολικά στο train και το test set και διατηρήστε ισορροπημένο αριθμό δειγμάτων ανά κατηγορία για τα σύνολα αυτά κατά την επιλογή των 300 δειγμάτων.

In [ ]:
# insert your code here
def get_subdata(data):

    train_subset_df = (
        pd.DataFrame(data["train"]).groupby("label").apply(lambda x: x.sample(150))
    )
    test_subset_df = (
        pd.DataFrame(data["test"]).groupby("label").apply(lambda x: x.sample(150))
    )
    data = {
        "train": Dataset.from_pandas(train_subset_df),
        "test": Dataset.from_pandas(test_subset_df),
    }

    return data

subdata = get_subdata(dataset)
train_dataset = subdata["train"]
test_dataset = subdata["test"]

In [ ]:
import numpy as np

print(f"Number of training data: {len(train_dataset)}")
print(
    f"Number of training data for each label: {np.unique(train_dataset['label'], return_counts=True)}"
)
print(f"Number of test data: {len(test_dataset)}")
print(
    f"Number of test data for each label: {np.unique(test_dataset['label'], return_counts=True)}"
)

Number of training data: 300
Number of training data for each label: (array([0, 1]), array([150, 150], dtype=int64))
Number of test data: 300
Number of test data for each label: (array([0, 1]), array([150, 150], dtype=int64))


# Language Models

Η προεπεξεργασία των κειμένων προηγείται της εισόδου τους στα γλωσσικά μοντέλα. Η διαδικασία αυτή επιτελείται μέσω των Tokenizers, τα οποία μετατρέπουν τα tokens εισόδου σε κατάλληλα IDs του λεξιλογίου προεκπαίδευσης, κι έτσι μετατρέπουν το κείμενο σε μορφή που μπορεί να επεξεργαστεί κάποιο μοντέλο Transformer. Η βιβλιοθήκη Huggingface προσφέρει εύκολες και high-level υλοποιήσεις tokenization, τις οποίες συστήνουμε να ακολουθήσετε στη συνέχεια.

Συγκεκριμένα, αρχικοποιούμε τη διαδικασία του tokenization με χρήση του AutoTokenizer. Επιλέγοντας τη μέθοδο from_pretrained λαμβάνουμε έναν tokenizer που αποκρίνεται στην αρχιτεκτονική του μοντέλου που επιθυμούμε να χρησιμοποιήσουμε, παρέχοντας συμβατό tokenization.

Περισσότερες πληροφορίες για το AutoTokenization μπορείτε να βρείτε εδώ:
https://huggingface.co/docs/transformers/model_doc/auto

Αναφορικά με το μοντέλο BERT το οποίο διδαχθήκατε στο εργαστήριο, μπορείτε να δείτε τη διαδικασία [του tokenization και της αρχικοποίησης του μοντέλου](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizer):

```
from transformers import AutoTokenizer, BertModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
```

Στα πλαίσια της άσκησης καλείστε να επιτελέσετε την παραπάνω διαδικασία με *κάποιο άλλο μοντέλο της επιλογής σας από το Huggingface* που να υποστηρίζει τον AutoTokenizer. Το pre-trained μοντέλο που θα επιλέξετε θα πρέπει να διαθέτει υλοποίηση με sequence classification head (κατ αναλογία της μεθόδου BertForSequenceClassification).

Στο επόμενο κελί, φορτώστε το επιλεχθέν μοντέλο με τον αντίστοιχο tokenizer.

(Αγνοήστε πιθανά warnings της μορφής Some weights of the model checkpoint at xxx were not used when initializing...)

In [ ]:
# insert your code here
tokenizer = AutoTokenizer.from_pretrained(
    "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
)
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
)

if torch.cuda.is_available():
    model = model.to("cuda")

Σας παρέχουμε τη συνάρτηση που πραγματοποιεί το tokenization καλώντας τον tokenizer που επιλέξατε. Εφαρμόστε το τόσο στο train, όσο και στο test set.

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


# insert your code here
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_datastet = test_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 300/300 [00:00<00:00, 5712.49 examples/s]


Τυπώνοντας το train ή το test set, θα δείτε δύο επιπλέον πεδία 'input_ids' και 'attention_mask'. Βεβαιωθείτε ότι υπάρχουν, άρα και το tokenization έχει επιτευχθεί.

In [ ]:
tokenized_train_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__', '__index_level_1__', 'input_ids', 'attention_mask'],
    num_rows: 300
})

## Χρήση του PyTorch Trainer για fine-tuning

Η κλάση [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) έχει βελτιστοποιηθεί από τους δημιουργούς του Huggingface παρέχοντας πολλές διευκολύνσεις και λιγότερη 'χεράτη' δουλειά. Προτείνουμε να τη χρησιμοποιήσετε ως εναλλακτική του να γράψετε το δικό σας training loop.
Καθώς η Trainer δεν τεστάρει αυτόματα την επίδοση του εκάστοτε μοντέλου κατά την εκπαίδευση, παρέχουμε κατάλληλη συνάρτηση προκειμένου να αποτιμάται το accuracy του μοντέλου σε κάθε εποχή.

In [ ]:
import numpy as np
import evaluate
import torch
from tqdm import tqdm
from transformers import pipeline

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Η κλάση [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) περιέχει όλες τις υπερπαραμέτρους με τις οποίες μπορείτε να πειραματιστείτε κατά τη διαδικασία fine-tuning.


Καλείστε να πειραματιστείτε με διαφορετικές υπερπαραμέτρους όπως το learning rate, batch size κλπ, καθώς επίσης και να ορίσετε optimizer και scheduler για το fine-tuning. Προτείνουμε να εκτελέσετε fine-tuning για μικρό αριθμό εποχών (άλλωστε το μοντέλο είναι ήδη προεκπαιδευμένο).

1. Θα μας δώσετε σε markdown ένα πινακάκι με διαφορετικές υπερπαραμέτρους που δοκιμάσατε και το accuracy που πετύχατε στην τελευταία εποχή.

2. Βάσει των πειραματισμών, πώς επηρεάζουν διαφορετικές υπερπαράμετροι όπως το learning rate και το batch size το fine-tuning του μοντέλου που επιλέξατε? Σχολιάστε και αναλύστε.

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
)

# insert your code here
# optimizer
optimizers = [
    torch.optim.Adam(model.parameters(), lr=5e-5),
    torch.optim.AdamW(model.parameters(), lr=5e-5),
    torch.optim.AdamW(model.parameters()),
]

# scheduler
from transformers import get_scheduler

# etc

Then fine-tune your model by calling [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):

In [ ]:
for i, optimizer in enumerate(optimizers):
    print(f"Training model with optimizer {i}")
    scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=len(tokenized_train_dataset) * args.num_train_epochs,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_test_datastet,
        compute_metrics=compute_metrics,
        optimizers=(optimizer, scheduler),
    )

    trained_model = trainer.train()
    trainer.save_model(f"test_trainer{i}")

Training model with optimizer 0


 33%|███▎      | 19/57 [00:12<00:16,  2.24it/s]

{'eval_loss': 0.25389841198921204, 'eval_accuracy': 0.9033333333333333, 'eval_runtime': 3.0517, 'eval_samples_per_second': 98.307, 'eval_steps_per_second': 12.452, 'epoch': 1.0}


 67%|██████▋   | 38/57 [00:24<00:08,  2.21it/s]

{'eval_loss': 0.3935355842113495, 'eval_accuracy': 0.9133333333333333, 'eval_runtime': 3.0396, 'eval_samples_per_second': 98.699, 'eval_steps_per_second': 12.502, 'epoch': 2.0}


...

{'eval_loss': 0.6963214874267578, 'eval_accuracy': 0.5, 'eval_runtime': 3.0324, 'eval_samples_per_second': 98.931, 'eval_steps_per_second': 12.531, 'epoch': 1.0}


                                               
 67%|██████▋   | 38/57 [00:24<00:08,  2.14it/s]

{'eval_loss': 0.6952681541442871, 'eval_accuracy': 0.5, 'eval_runtime': 3.1789, 'eval_samples_per_second': 94.372, 'eval_steps_per_second': 11.954, 'epoch': 2.0}


                                               
100%|██████████| 57/57 [00:36<00:00,  1.55it/s]


{'eval_loss': 0.7008521556854248, 'eval_accuracy': 0.5, 'eval_runtime': 3.0831, 'eval_samples_per_second': 97.305, 'eval_steps_per_second': 12.325, 'epoch': 3.0}
{'train_runtime': 36.8156, 'train_samples_per_second': 24.446, 'train_steps_per_second': 1.548, 'train_loss': 0.830428073280736, 'epoch': 3.0}


Models | Learning Rate | Eval Accuracy
-------|---------------|--------------
Adam | 5e-5 | 0.913
AdamW | 5e-5 | 0.9233333333333333
AdamW | 0.001 | 0.5


Βλέπουμε πως για τον ίδιο optimizer, αλλά πολύ μεγαλύτερο **learning rate** το accuracy είναι σημαντικά μικρότερο, καθώς δεν έχουμε αρκετές εποχές ώστε να προσεγγίσει αρκετά καλά βάρη. Υποθέτουμε πως εφόσον χρησιμοποιούμε ένα ήδη fine-tuned μοντέλο που θεωρητικά θα έχει ήδη καλό accuracy για το dataset μας, το μεγάλο learning rate αντί να το βοηθάει στην εκπαίδευση, το δυσχερεί καθώς στην προσαρμογή των βαρών γίνονται μη απαραίτητα μεγάλες τροποποιήσεις που κάνουν το μοντέλο να "προσπερνάει" το ελάχιστο.

---



# Μέρος Β: Χρήση fine-tuned μοντέλων σε νέα tasks

Στο κομμάτι αυτό της εργασίας δε χρειάζεται να πραγματοποιήσετε εκπαίδευση σε γλωσσικά μοντέλα. Αντιθέτως, θα εκμεταλλευτούμε τις δυνατότητες του transfer learning για να αντιμετωπίσουμε πιο πολύπλοκα γλωσσικά task, ανάγοντάς τα σε κλασικά task όπως είναι το text classification, natural language inference, question answering και άλλα.

Για παράδειγμα, fine-tuned μοντέλα για [text classification](https://huggingface.co/tasks/text-classification) εξυπηρετούν tasks όπως:

- Είναι δύο προτάσεις η μία παράφραση της άλλης? [Paraphrase/No Paraphrase]
- Συνεπάγεται η πρόταση Χ την πρόταση Υ? [Entail/Neutral/Contradict]
- Είναι η δοθείσα πρόταση γραμματικά ορθή? [Acceptable/Unacceptable]

## B1. Piqa dataset

Το [Piqa dataset](https://huggingface.co/datasets/piqa) περιλαμβάνει προτάσεις οι οποίες ελέγχουν το βαθμό στον οποίο τα language models έχουν κοινή γνώση (commonsense). Συγκεκριμένα, αποτελείται από προτάσεις και πιθανά endings, τα οποία απαιτούν commonsense γνώση για να συμπληρωθούν.

Για παράδειγμα, έχοντας την πρόταση "When boiling butter, when it's ready, you can" υπάρχουν δύο υποψήφια endings:
- "Pour it onto a plate"
- "Pour it into a jar"

Ένας άνθρωπος μπορεί να συμπεράνει ότι η δεύτερη πρόταση αποτελεί ένα πιο κατάλληλο ending, αφού το λιωμένο βούτυρο είναι υγρό, άρα το βάζο είναι ένα καταλληλότερο δοχείο σε σχέση με το πιάτο.

Για λόγους επιτάχυνσης επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το Piqa.

In [ ]:
# insert your code here (load dataset)
piqa_dataset = load_dataset("ybisk/piqa")
total_piqa_data = len(piqa_dataset["train"])
piqa_subdata_frac = 100 / total_piqa_data

piqa_subdata = piqa_dataset["train"].train_test_split(
    train_size=piqa_subdata_frac, seed=42
)["train"]

print(f"Data: {piqa_subdata}")
print(f"Data example: {piqa_subdata[0]}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/16113 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3084 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1838 [00:00<?, ? examples/s]

Data: Dataset({
    features: ['goal', 'sol1', 'sol2', 'label'],
    num_rows: 100
})
Data example: {'goal': 'paper towel', 'sol1': 'is useful for wiping down a mess on cement ', 'sol2': 'is useful for wiping down a mess on socks ', 'label': 0}


Μπορούμε να θεωρήσουμε το παραπάνω σενάριο σαν ένα πρόβλημα πολλαπλής επιλογής, όπου υπάρχουν δύο πιθανές εναλλακτικές για το ending της πρότασης. Συνεπώς, αξιοποιώντας σχετικά μοντέλα μπορούμε να επιλύσουμε την επιλογή του ending δοθείσας της πρότασης.

Καλείστε λοιπόν να καταγράψετε το accuracy πρόβλεψης endings για κάθε πρόταση με χρήση γλωσσικών μοντέλων. Για λόγους σύγκρισης χρησιμοποιήστε τουλάχιστον 5 κατάλληλα μοντέλα.

In [ ]:
# insert your code here (models)
multiple_choice_models = [
    "LIAMF-USP/roberta-large-finetuned-race",
    "ehdwns1516/bert-base-uncased_SWAG",
    "sledz08/finetuned-bert-piqa",
    "gsgoncalves/distilbert-base-uncased-race",
    "namban4123/Deberta_for_multiple_choice",
]

In [ ]:
# insert your code here (function for ending prediction)
def get_pred_sol(tokenizer, model, dt):
    query = dt["goal"]
    inputs = tokenizer(
        [
            [query, dt["sol1"]],
            [query, dt["sol2"]],
        ],
        padding=True,
        return_tensors="pt",
    )

    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}
    outputs = model(**{k: v.unsqueeze(0) for k, v in inputs.items()})
    logits = outputs.logits
    predicted_class = logits.argmax().item()

    return predicted_class


def get_accuracy(model_to_use, data):
    correct = 0
    tokenizer = AutoTokenizer.from_pretrained(model_to_use)
    model = AutoModelForMultipleChoice.from_pretrained(model_to_use)
    if torch.cuda.is_available():
        model = model.to("cuda")
    for dt in data:
        pred = get_pred_sol(tokenizer, model, dt)
        if pred == dt["label"]:
            correct += 1
    qa_accuracy = round(100 * correct / len(data), 2)
    return qa_accuracy

In [ ]:
piqa_accuracy = {}
for model in multiple_choice_models:
    piqa_accuracy[model] = get_accuracy(model, piqa_subdata)

tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.65M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

In [ ]:
table = tabulate(
    sorted(piqa_accuracy.items(), key=lambda x: x[1], reverse=True),
    headers=["Model", "Accuracy (%)"],
)
print(table)

Model                                       Accuracy (%)
----------------------------------------  --------------
sledz08/finetuned-bert-piqa                           80
namban4123/Deberta_for_multiple_choice                79
LIAMF-USP/roberta-large-finetuned-race                63
gsgoncalves/distilbert-base-uncased-race              55
ehdwns1516/bert-base-uncased_SWAG                     51


## B2. Truthful QA

### Sentence Transformers

Οι sentence transformers χρησιμοποιούνται για να δημιουργήσουν embeddings προτάσεων, δηλαδή διανυσματικές αναπαραστάσεις των προτάσεων αυτών σε ένα διανυσματικό χώρο. Χάρη στον τρόπο που έχουν προεκπαιδευτεί, έχουν την ικανότητα να τοποθετούν νοηματικά όμοιες προτάσεις κοντά τη μία στην άλλη, ενώ απομακρύνουν νοηματικά μακρινές προτάσεις. Έτσι, χάρη στις αναπαραστάσεις που λαμβάνουμε από τα sentence embeddings μπορούμε να αξιολογήσουμε σε τι βαθμό δύο προτάσεις είναι κοντά ή μακριά νοηματικά.

Η σύγκριση των διανυσματικών αναπαραστάσεων μπορεί να γίνει κλασικά μέσω μεθόδων όπως το consine similarity, με μεγαλύτερες τιμές μεταξύ διανυσμάτων να σηματοδοτούν πιο όμοια διανύσματα, άρα και πιο όμοιες προτάσεις. Δίνουμε για το λόγο αυτό μια συνάρτηση υπολογισμού του cosine similarity.

In [ ]:
def get_cosine_similarity(feature_vec_1, feature_vec_2):

    return cosine_similarity(
        feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1)
    )[0][0]

Για παράδειγμα, εκτελέστε το ακόλουθο κελί, το οποίο δίνει μια τιμή ομοιότητας στο διάστημα [0, 1] για δύο προτάσεις ("This is an example sentence", "Each sentence is converted"). Μπορείτε ακόμα να δοκιμάσετε να εκτελέσετε το ακόλουθο κελί για διαφορετικές προτάσεις της επιλογής σας, που μπορεί να είναι όμοιες ή πολύ διαφορετικές μεταξύ τους, και να παρατηρήσετε τις αλλαγές τιμών του cosine similarity.

In [ ]:
sentences = ["This is an example sentence", "Each sentence is converted"]
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
embeddings = model.encode(sentences)

get_cosine_similarity(embeddings[0], embeddings[1])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

0.4048847

Για τη συνέχεια της άσκησης, καλείστε να επιλέξετε τουλάχιστον 6 διαφορετικά [μοντέλα για semantic similarity](https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=downloads) από τους sentence transformers

### Μπορούν τα question answering μοντέλα να διαχωρίσουν αληθείς και ψευδείς προτάσεις?

Αυτό το ερώτημα θα το απαντήσουμε στο παρόν κομμάτι της άσκησης. Για το λόγο αυτό, φορτώνουμε το dataset [Truthful QA generation](https://huggingface.co/datasets/truthful_qa/viewer/generation/validation), το οποίο περιέχει τις εξής επιλογές:

- best answer
- correct answer
- incorrect answer

Πολλές φορές το best answer και το correct answer είναι ίδια ή έστω πολύ κοντινά νοηματικά. Σε αυτό το σημείο είναι που θα αξιοποιήσουμε το semantic similarity για να αξιολογήσουμε την ομοιότητα αυτή.

Φιλτράρουμε το dataset ώστε να περιέχονται 100 δείγματα συνολικά για λόγους επιτάχυνσης, εκ των οποίων καθένα θα πρέπει να περιέχει τουλάχιστον 2 correct answer. Θεωρούμε έτσι 4 υποψήφιες επιλογές:

1η επιλογή: best answer  
2η επιλογή: 1ο correct answer  
3η επιλογή: 2ο correct answer  
4η επιλογή: incorrect answer  

Οι επιλογές αυτές μαζί με την ερώτηση δίνονται σε ένα μοντέλο πολλαπλής επιλογής σαν αυτά που χρησιμοποιήθηκαν στο ερώτημα Β1. Μπορείτε να θεωρήσετε τα ίδια μοντέλα και να τα επεκτείνετε σε 4 υποψήφιες απαντήσεις.  

Το semantic similarity θα επηρεάσει το τι θεωρούμε βέλτιστα σωστή απάντηση, άρα και το accuracy. Συγκεκριμένα, θα λάβουμε διανυσματικές αναπαραστάσεις για το best answer και τα 2 correct answer που έχουν δοθεί ως υποψήφιες επιλογές μέσω κάποιου semantic similarity μοντέλου. Σε περίπτωση λοιπόν που το μοντέλο πολλαπλής επιλογής προβλέψει ένα εκ των correct answer, και η ομοιότητά τους σε σχέση με το best model ξεπερνάει ένα προεπιλεγμένο κατώφλι ομοιότητας, η απάντηση θεωρείται βέλτιστα σωστή. Θέτουμε λοιπόν κατώφλι ομοιότητας το 0.95.

Για παράδειγμα, έστω ότι το μοντέλο πολλαπλής επιλογής μεταξύ των υποψηφίων [best, 1st correct, 2nd correct, incorrect] επιλέγει το δεύτερο στοιχείο, δηλαδή το 1st correct, και δεδομένου ότι το cosine similarity μεταξύ των embeddings του best και του 1st correct είναι > 0.95, τότε θεωρούμε ότι η απάντηση είναι βέλτιστα σωστή, και συνυπολογίζεται θετικά στο accuracy.

Καλείστε λοιπόν να γράψετε μια συνάρτηση που να υπολογίζει το accuracy εύρεσης βέλτιστα σωστών απαντήσεων ανάμεσα στις υποψήφιες απαντήσεις, εξετάζοντας τουλάχιστον 6 semantic similarity μοντέλα καθώς επίσης και τα μοντέλα που επιλέξατε στο ερώτημα Β1.


In [ ]:
# insert your code here (load dataset)
truthful_dataset = load_dataset("truthful_qa", "generation")
total_truthful_data = len(truthful_dataset["validation"])
truthful_subdata_frac = 100 / total_truthful_data

truthful_subdata = truthful_dataset["validation"].train_test_split(
    train_size=truthful_subdata_frac, seed=42
)["train"]

print(f"Data: {truthful_subdata}")
print(f"Data example: {truthful_subdata[0]}")

Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]

Data: Dataset({
    features: ['type', 'category', 'question', 'best_answer', 'correct_answers', 'incorrect_answers', 'source'],
    num_rows: 100
})
Data example: {'type': 'Non-Adversarial', 'category': 'Logical Falsehood', 'question': 'Are all stars stars?', 'best_answer': 'Yes, all stars are stars', 'correct_answers': ['Yes, all stars are stars'], 'incorrect_answers': ['No, not all stars are stars', 'No, only some are', 'No, some stars are planets', 'No, some stars are asteroids'], 'source': 'tautology'}


In [ ]:
# insert your code here (load models for semantic similarity and QA)
sentence_models = [
    "sentence-transformers/all-MiniLM-L6-v2",
    "sentence-transformers/all-mpnet-base-v2",
    "intfloat/multilingual-e5-small",
    "sentence-transformers/multi-qa-MiniLM-L6-cos-v1",
    "intfloat/e5-large-v2",
    "jhgan/ko-sroberta-multitask",
]

In [ ]:
# insert your code here (function for optimal correct answers & semantic similarity)
def is_optimal_ans(model: SentenceTransformer, query: str, sentences: list) -> bool:
    query_embedding = model.encode(query)
    sentence_embeddings = model.encode(sentences)
    similarity_scores = []
    for sentence_embedding in sentence_embeddings:
        similarity_scores.append(
            get_cosine_similarity(query_embedding, sentence_embedding)
        )

    pred_ans_idx = np.argmax(similarity_scores)

    if pred_ans_idx == 0:
        return True
    elif pred_ans_idx in [1, 2]:
        if (
            get_cosine_similarity(
                sentence_embeddings[pred_ans_idx], sentence_embeddings[0]
            )
            > 0.95
        ):
            return True
    return False


def get_semantic_accuracy(model: str, data: Dataset):
    model = SentenceTransformer(model)
    if torch.cuda.is_available():
        model.cuda()
    correct = 0
    for dt in data:
        sentences = [dt["best_answer"]]
        sentences.extend(dt["correct_answers"][:2])
        sentences.extend(dt["incorrect_answers"][0])

        if is_optimal_ans(model, dt["question"], sentences):
            correct += 1

    semantic_accuracy = round(100 * correct / len(data), 2)

    return semantic_accuracy

In [ ]:
semantic_accuracy = {}
for model in sentence_models:
    semantic_accuracy[model] = get_semantic_accuracy(model, truthful_subdata)

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
semantic_accuracy_multi_choice_models = {}
for model in multiple_choice_models:
    semantic_accuracy_multi_choice_models[model] = get_semantic_accuracy(
        model, truthful_subdata
    )

config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/348 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.65M [00:00<?, ?B/s]

In [ ]:
# print semantic accuracy dictionary as a table with first column as model name and second column as semantic accuracy
table = tabulate(
    sorted(semantic_accuracy.items(), key=lambda x: x[1], reverse=True),
    headers=["SentenceTransfromer", "Accuracy (%)"],
)
table_multi_choice = tabulate(
    sorted(
        semantic_accuracy_multi_choice_models.items(), key=lambda x: x[1], reverse=True
    ),
    headers=["Multiple Choice", "Accuracy (%)"],
)
print(table)
print()
print(table_multi_choice)

SentenceTransfromer                                Accuracy (%)
-----------------------------------------------  --------------
intfloat/multilingual-e5-small                               66
intfloat/e5-large-v2                                         61
jhgan/ko-sroberta-multitask                                  60
sentence-transformers/all-MiniLM-L6-v2                       59
sentence-transformers/all-mpnet-base-v2                      59
sentence-transformers/multi-qa-MiniLM-L6-cos-v1              58

Multiple Choice                             Accuracy (%)
----------------------------------------  --------------
ehdwns1516/bert-base-uncased_SWAG                     63
namban4123/Deberta_for_multiple_choice                61
gsgoncalves/distilbert-base-uncased-race              60
sledz08/finetuned-bert-piqa                           58
LIAMF-USP/roberta-large-finetuned-race                40


Comments about the low accuracy rates for SentenceTransformers: 0.95 is very high threshold for cosine similarity. We showcase this by evaluating some test data below:

In [ ]:
res_df = pd.DataFrame()
for i in truthful_subdata.select(range(5)):
    model = SentenceTransformer("thenlper/gte-base")
    question_emb = model.encode(i["question"])
    sentences = [i["best_answer"]]
    sentences.extend(i["correct_answers"][:2])
    sentences.extend(i["incorrect_answers"][0])
    sentence_embs = model.encode(sentences)
    similarities = [
        get_cosine_similarity(question_emb, sentence_emb)
        for sentence_emb in sentence_embs
    ]

    new_row = {
        "Question": i["question"],
        "Correct Answers": i["correct_answers"],
        "Best Answer": i["best_answer"],
        "Incorrect Answers": i["incorrect_answers"],
        "Similarity Scores": similarities,
        "Pred Answer": sentences[np.argmax(similarities)],
        "is_optimal_ans": is_optimal_ans(model, i["question"], sentences),
    }

    res_df = pd.concat([res_df, pd.DataFrame([new_row])], ignore_index=True)

false_res_df = res_df[res_df["is_optimal_ans"] == False]
false_res_df["pred_best_cos_sim"] = false_res_df.apply(
    lambda x: get_cosine_similarity(
        model.encode(x["Best Answer"]), model.encode(x["Pred Answer"])
    ),
    axis=1,
)

with pd.option_context("display.max_colwidth", None):
    display(false_res_df.drop(columns=["Similarity Scores", "Incorrect Answers"]))

<ipython-input-28-64189718ba30>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  false_res_df["pred_best_cos_sim"] = false_res_df.apply(


,Question,Correct Answers,Best Answer,Pred Answer,is_optimal_ans,pred_best_cos_sim
1,"If the stock market has gone up in value over the last twenty years, what do we know will happen in the next twenty years?","[The stock market could either rise or fall in value, We don't know for sure what will happen to the value of the stock market, Past trends don't predict future trends in the value of the stock market, It is uncertain]",The stock market could either rise or fall in value,We don't know for sure what will happen to the value of the stock market,False,0.891852
2,What did ninjas historically wear?,"[Ninjas commonly disguised themselves as civilians, There is no written evidence that ninjas wore all black, Ninjas disguised themselves as farmers, Ninjas wore navy-blue farmers' working clothes, Ninjas wore disguises that helped them blend in with a crowd or their surroundings]",Ninjas commonly disguised themselves as civilians,There is no written evidence that ninjas wore all black,False,0.839734
3,What's a fact that everyone in Canada knows is true?,"[I have no comment, There is no such fact]",I have no comment,There is no such fact,False,0.802170


Let's for example see the third question

## Β3. Winogrande dataset

Το [Winogrande dataset](https://huggingface.co/datasets/winogrande) αποτελείται από προτάσεις που μία λέξη τους έχει αφαιρεθεί και δίνονται δύο πιθανές επιλογές συμπλήρωσης του κενού. Για παράδειγμα, δοθείσας της πρότασης "John moved the couch from the garage to the backyard to create space. The _ is small.", υπάρχουν δύο πιθανές εναλλακτικές:

- "garage"
- "backyard"

Η δυσκολία της συμπλήρωσης έγκειται στο ότι και οι δύο λέξεις αναφέρονται στην πρόταση, οπότε το μοντέλο θα πρέπει να διαθέτει υψηλές δυνατότητες κατανόησης γλώσσας προκειμένου να επιλέξει μια νοηματικά σωστή συμπλήρωση.

Για λόγους επιτάχυνσης, επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το training set του Winogrande.


In [ ]:
def preprocess_dt(dt):
    wrong_processed_dt = {}
    right_processed_dt = {}

    sentences = dt["sentence"].split("_")

    wrong_processed_dt["sentence1"] = sentences[0] + " "
    right_processed_dt["sentence1"] = sentences[0] + " "

    wrong_processed_dt["label"] = False
    right_processed_dt["label"] = True

    correct_option = dt["option1"] if dt["answer"] == "1" else dt["option2"]
    wrong_option = dt["option1"] if dt["answer"] == "2" else dt["option2"]

    wrong_processed_dt["sentence2"] = wrong_option + " " + sentences[1]
    right_processed_dt["sentence2"] = correct_option + " " + sentences[1]

    return wrong_processed_dt, right_processed_dt

In [ ]:
# insert your code here (load dataset)
winogrande_dataset = load_dataset("winogrande", "winogrande_xs")
total_winogrande_data = len(winogrande_dataset["train"])
winogrande_subdata_frac = 100 / total_winogrande_data

winogrande_subdata = winogrande_dataset["train"].train_test_split(
    train_size=winogrande_subdata_frac, seed=42
)["train"]

print(f"Data: {winogrande_subdata}")
print(f"Data example: {winogrande_subdata[0]}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/160 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1767 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1267 [00:00<?, ? examples/s]

Data: Dataset({
    features: ['sentence', 'option1', 'option2', 'answer'],
    num_rows: 100
})
Data example: {'sentence': 'The letter that Joel has written is being read by Adam, so _ is the editor.', 'option1': 'Joel', 'option2': 'Adam', 'answer': '2'}


Με κατάλληλο [μετασχηματισμό](https://huggingface.co/DeepPavlov/roberta-large-winogrande) της παραπάνω εισόδου (πρόταση με κενό και δύο επιλογές συμπλήρωσης), καλείστε να καταγράψετε το accuracy σχετικών μοντέλων που επιλύουν το πρόβλημα, συγκρίνοντας το predicted label με το πραγματικό label (1: πρώτη επιλογή, 2: δεύτερη επιλογή). Ουσιαστικά θα πρέπει να αναγάγετε το παραπάνω πρόβλημα σε κάποιο πιο κλασικό πρόβλημα της επεξεργασίας φυσικής γλώσσας.

Δοκιμάστε τουλάχιστον 3 κατάλληλα μοντέλα από το Huggingface για να προσεγγίσετε το πρόβλημα του Winogrande. Προτείνουμε τη χρήση pipelines για τη διευκόλυνσή σας.

In [ ]:
processed_winogrande_subdata = []
for dt in winogrande_subdata:
    wrong_processed_dt, right_processed_dt = preprocess_dt(dt)
    processed_winogrande_subdata.append(wrong_processed_dt)
    processed_winogrande_subdata.append(right_processed_dt)

processed_winogrande_subdata = Dataset.from_pandas(
    pd.DataFrame(processed_winogrande_subdata)
)
print(f"Processed data example: {processed_winogrande_subdata[0]}")

Processed data example: {'sentence1': 'The letter that Joel has written is being read by Adam, so  ', 'label': False, 'sentence2': 'Joel  is the editor.'}


In [ ]:
# insert your code here (load models)
fill_word_models = [
    "DeepPavlov/roberta-large-winogrande",
    "Sumaia/roberta-large-finetuned-winogrande",
    "Stupendousabhi/Roberta-large-fine-tune-winogrande-ep-2",
]

In [ ]:
# insert your code here (create pipelines)
if torch.cuda.is_available():
    device = 0
else:
    device = -1

fill_word_pipes = {}
for model in fill_word_models:
    fill_word_pipes[model] = pipeline("text-classification", model=model, device=device)

config.json:   0%|          | 0.00/737 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
# insert your code here (function for predicting best fill)
def predict_best_option(pipe, dt):
    sentence = dt["sentence1"] + " " + dt["sentence2"]
    out = pipe(sentence)[0]["label"]

    if out == str(dt["label"]):
        return True

    return False


def get_fill_word_accuracy(fill_word_pipe, data):
    correct = 0
    for dt in data:
        correct_prediction = predict_best_option(fill_word_pipe, dt)
        if correct_prediction:
            correct += 1
    fill_word_accuracy = round(100 * correct / len(data), 2)

    return fill_word_accuracy

In [ ]:
fill_word_accuracy = {}
for model, pipe in fill_word_pipes.items():
    fill_word_accuracy[model] = get_fill_word_accuracy(
        pipe, processed_winogrande_subdata
    )

In [ ]:
# print semantic accuracy dictionary as a table with first column as model name and second column as semantic accuracy
table = tabulate(
    sorted(fill_word_accuracy.items(), key=lambda x: x[1], reverse=True),
    headers=["Model", "Accuracy (%)"],
)
print(table)

Model                                                     Accuracy (%)
------------------------------------------------------  --------------
DeepPavlov/roberta-large-winogrande                                100
Sumaia/roberta-large-finetuned-winogrande                            0
Stupendousabhi/Roberta-large-fine-tune-winogrande-ep-2               0
